# Start Gui

This notebook can be used to start the GUI and test out stuff.

In [31]:
import randomizer_modules as rm


In [1]:
# -*- coding: utf-8 -*-
"""
@author: Fabian Balzer

***
LICENSE:
    Copyright 2021 Fabian Balzer

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
***

Code to open a GUI Dominion randomizer
"""
# %%

import sys

import PyQt5.QtWidgets as QW
from randomizer_modules.main_ui import UIMainWindow


def start_program():
    """A function to include everything needed to start the application"""
    # Check whether there is already a running QApplication (e.g. if running
    # from an IDE). This setup prevents crashes for the next run:
    qapp = QW.QApplication.instance()
    if not qapp:
        qapp = QW.QApplication(sys.argv)
    app = UIMainWindow()  # creating the instance
    app.show()
    app.activateWindow()
    qapp.exec_()  # Start the Qt event loop



In [9]:
start_program()

In [ ]:
df = rm.read_dataframe_from_file(rm.FPATH_CARD_DATA, True)

type(df["attack_types"].iloc[0])
path = rm.PATH_MAIN.joinpath(df.iloc[0]["ImagePath"])
df.set_index("Name").loc["Young Witch"]["Text"]
import pandas as pd
selection = ["Way", "Ally"]
kingdom = rm.Kingdom(df, rm.get_randomizer_config_options())
kingdom.randomize([])
kingdom.get_csv_representation(), kingdom.special_targets


In [70]:
import numpy as np
def _calculate_total_quality(values):
    # Initialize array with zeros representing 0, 1, 2, 3, 4
    counts = np.zeros(5)
    
    # If there are 5-i values of the same thing, increment the value count of the next one
    # e.g. a list of [1, 1, 1, 1, 2] should be counted as two 2s. because there are four
    # ones, three 2s will be counted as a single 3, and two 3s will yield a 4.
    for i in range(4):
        counts[i] += values.count(i)
        if i == 0:
            continue
        counts[i+1] += counts[i] // (5-i)
    
    # Look where the first nonzero value sits.
    total_quality_value = np.nonzero(counts)[0][-1]
    
    return total_quality_value

qual = "village"
values = kingdom.get_kingdom_df()[qual + "_quality"].to_list()
print(_calculate_total_quality(values), values)
kingdom.get_kingdom_df()[["Name", f"{qual}_quality"]]

3 [2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0]


,Name,village_quality
277,Page,2
41,Pawn,0
603,Sycophant,0
773,Augurs,0
215,Sage,0
680,Siren,0
47,Swindler,0
54,Ironworks,0
290,Port,3
65,Torturer,0


In [66]:
path = 'C:/Users/fabia/OneDrive/Dokumente/Freizeit/Dominion/Dominion_randomizer/card_pictures/Base/Cellar.jpg'
import sys
import randomizer_modules as rm
df = rm.read_dataframe_from_file(rm.FPATH_CARD_DATA, True)
import numpy as np
from randomizer_modules.base_widgets import CoolButton
import PyQt5.QtCore as QC
import PyQt5.QtWidgets as QW
from PyQt5.QtGui import QPixmap, QPainter, QImage, QImageReader, QPainterPath, QPolygon, QColor, QIcon
import pandas as pd

        
class ScrollableGroupBox(QW.QGroupBox):
    def __init__(self, title, parent=None):
        super().__init__(title, parent)
        self.init_ui()

    def init_ui(self):
        layout = QW.QVBoxLayout(self)

        scroll_area = QW.QScrollArea(self)
        layout.addWidget(scroll_area)

        scroll_content = QW.QWidget(scroll_area)
        scroll_area.setWidget(scroll_content)

        self.content_layout = QW.QVBoxLayout(scroll_content)

        scroll_area.setWidgetResizable(True)
        scroll_area.setVerticalScrollBarPolicy(QC.Qt.ScrollBarAsNeeded)
        scroll_area.setHorizontalScrollBarPolicy(QC.Qt.ScrollBarAlwaysOff)
    
    def addWidget(self, widget: QW.QWidget, stretch=None):
        self.content_layout.addWidget(widget, stretch)



class LandscapeDisplay(QW.QWidget):
    """Display Name, Picture and Image for the given kingdom card"""

    def __init__(self, landscape: pd.Series, width=360):
        super().__init__()
        impath = str(rm.PATH_MAIN.joinpath(landscape["ImagePath"]))
        image_wid = rm.ImageCutoutWidget(impath, 0.05, 1, width)

        lay = QW.QVBoxLayout(self)
        lay.addWidget(image_wid)
        lay.setSpacing(0)
        lay.setAlignment(QC.Qt.AlignTop)
        lay.setContentsMargins(0, 0, 0, 0)
        lay.setSizeConstraint(QW.QVBoxLayout.SetMinimumSize)

        self.setSizePolicy(QW.QSizePolicy.Fixed, QW.QSizePolicy.Fixed)


        
        # Create a frame widget to act as the rectangular frame
        label = QW.QLabel(landscape.Name, self)
        label.setStyleSheet("border-image: url('demo.jpg');\
                            background-color: gray; \
                            border-radius:50")
        label.setMargin(8)
        label.setScaledContents(True)
        label.setAlignment(QC.Qt.AlignCenter)
        height = image_wid.label.pixmap().height()
        font = label.font()
        font.setPointSize(16)
        label.setFont(font)
        label.setGeometry(12, height - 50, width - 24, 50)
        self.label = label

        # Create a button and add it to a widget
        button = rm.RerollButton(self)
        button.move(20, height - 50 + 12)
        self.reroll_button = button
        self.name = landscape.Name
        
        # # Create a frame widget to act as the rectangular frame
        self.setAutoFillBackground(True)
        palette = self.palette()
        palette.setColor(self.backgroundRole(), QC.Qt.black)
        self.setPalette(palette)


class MainWindow(QW.QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Scrollable QGroupBox")
        self.setGeometry(100, 100, 400, 300)

        self.central_widget = QW.QWidget(self)
        self.setCentralWidget(self.central_widget)

        layout = QW.QVBoxLayout(self.central_widget)

        wid = LandscapeDisplay(df[df.IsLandscape].iloc[0])
        layout.addWidget(wid)
        return
        scroll_group_box_1 = ScrollableGroupBox("Scrollable Group Box 1", self.central_widget)
        layout.addWidget(scroll_group_box_1)

        scroll_group_box_2 = ScrollableGroupBox("Scrollable Group Box 2", self.central_widget)
        layout.addWidget(scroll_group_box_2)




def start_app():
    qapp = QW.QApplication.instance()
    if not qapp:
        qapp = QW.QApplication(sys.argv)
    window = MainWindow()
    window.show()
    window.activateWindow()
    qapp.exec_()
start_app()
